## 处理一带一路相关国家的双边贸易数据

### 数据来源
* 中国一带一路网 https://www.yidaiyilu.gov.cn/jcsjpc.htm
* 中国经济信息网 一带一路统计数据库 https://ydyl.cei.cn/

### 数据处理
* 利用json读入贸易数据原始文件
* 将读入的字典数据按年份重新分割并保存
* 以年为维度对双边贸易进行可视化

### 分析与可视化过程
* 导入pyecharts包
* 导入所有国家的自定义经纬度坐标(coor.json文件)
* 使用pyecharts的Geo图表，以年为单位对历年中国对一带一路国家双边贸易数据进行可视化

In [1]:
import json
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Map, Geo, Timeline
from pyecharts.globals import ChartType, SymbolType

In [2]:
fn = 'data/一带一路相关数据/双边贸易.json'
with open(fn, 'r', encoding="utf-8") as f:
    trade = json.load(f)
trade = trade['total']
# print(trade)
_ = {}
year = 2013
for i in trade:
    _[str(year)] = []
    for k, v in i.items():
        if k != 'date':
            if v != '':
                _[str(year)].append((k, int(v)))
            else:
                _[str(year)].append((k, 0))
    year += 1
for i in range(2013, 2021):
    fn = 'data/一带一路相关数据/trade_data/trade_data_' + str(i) + '.json'
    with open(fn, 'w', encoding='utf-8') as f:
        json.dump(_[str(i)], f)


In [3]:
tl = Timeline()
for j in range(2013, 2021):
    fn = "data/一带一路相关数据/trade_data/trade_data_" + str(j) + ".json"
    with open(fn, 'r', encoding='utf-8') as f:
        data = json.load(f)
    countries = [i[0] for i in data]
    trade = []
    cost = [i[1] for i in data]
    rela = []
    for i in data:
        if i[1] > 0:
            rela.append(("中国", i[0]))
            trade.append(tuple(i))
    c = (
        Geo()
        .add_coordinate_json("data/coor.json")
        .add_schema(maptype="world")
        .add(
            "",
            trade,
            type_=ChartType.EFFECT_SCATTER
        )
        .add(
            "",
            rela,
            type_=ChartType.LINES,
            effect_opts=opts.EffectOpts(
                symbol=SymbolType.ARROW, symbol_size=5, color="red"
            ),
            linestyle_opts=opts.LineStyleOpts(curve=0.2, color='gold'),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=max(cost)), title_opts=opts.TitleOpts(title="{}年中国与一带一路亚非欧国家双边贸易可视化".format(j), subtitle="进出口总额\t单位：万美元"))
    )
    tl.add(c, "{}年".format(j))
tl.render_notebook()
